In [2]:
#Importing necessary libraries for data loading and manipulation
import pandas as pd
import numpy as np
import io
import re

In [4]:
#Loading data
df = pd.read_csv(r"C:\Users\Aneesh\Downloads\Pacta Assessment Project\aggregated data.csv")

In [6]:
#Inspecting the data
print("Initial Data Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())

Initial Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3309 entries, 0 to 3308
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State Name     3309 non-null   object 
 1   District Name  3309 non-null   object 
 2   Disability     3309 non-null   object 
 3   Age Group      3309 non-null   object 
 4   Male           2851 non-null   float64
 5   Female         2561 non-null   float64
 6   Other          119 non-null    float64
 7   Total          3013 non-null   float64
dtypes: float64(4), object(4)
memory usage: 206.9+ KB
None

Missing Values:
State Name          0
District Name       0
Disability          0
Age Group           0
Male              458
Female            748
Other            3190
Total             296
dtype: int64


In [8]:
# Standardizing column names: lowercase, replace spaces with underscores, remove special characters
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)

In [10]:
# Cleaning 'age_group' column: remove quotes and standardize format
df['age_group'] = df['age_group'].str.strip("'")

In [12]:
# Handling missing values in numeric columns: replace empty strings with 0
numeric_columns = ['male', 'female', 'other', 'total']
for col in numeric_columns:
    df[col] = df[col].fillna(0)  # Fill empty cells with 0

In [14]:
# Converting numeric columns to integers, ensuring float to integer conversion
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

In [16]:
# Validating 'total' column: check if total = male + female + other
df['calculated_total'] = df['male'] + df['female'] + df['other']
discrepancies = df[df['total'] != df['calculated_total']]
if not discrepancies.empty:
    print("\nRows with Total discrepancies:")
    print(discrepancies[['state_name', 'district_name', 'disability', 'age_group', 'male', 'female', 'other', 'total', 'calculated_total']])
    # Fix discrepancies by setting total to calculated_total
    df.loc[df['total'] != df['calculated_total'], 'total'] = df['calculated_total']

In [18]:
# Dropping the temporary 'calculated_total' column
df = df.drop(columns=['calculated_total'])

In [20]:
# Checking for duplicates and remove if any
duplicates = df.duplicated().sum()
if duplicates > 0:
    print(f"\nFound {duplicates} duplicate rows. Removing them.")
    df = df.drop_duplicates()

In [22]:
# Validating for negative or invalid values in numeric columns
invalid_rows = df[(df['male'] < 0) | (df['female'] < 0) | (df['other'] < 0) | (df['total'] < 0)]
if not invalid_rows.empty:
    print("\nRows with negative values:")
    print(invalid_rows)
    # Remove rows with negative values (if any)
    df = df[(df['male'] >= 0) & (df['female'] >= 0) & (df['other'] >= 0) & (df['total'] >= 0)]

In [24]:
# Standardizing text columns: trim whitespace and convert to title case for consistency
text_columns = ['state_name', 'district_name', 'disability']
for col in text_columns:
    df[col] = df[col].str.strip().str.title()

In [26]:
# Ensuring 'age_group' is treated as a categorical variable
df['age_group'] = df['age_group'].astype('category')

In [28]:
# Verifying data types after cleaning
print("\nData Types After Cleaning:")
print(df.dtypes)


Data Types After Cleaning:
state_name         object
district_name      object
disability         object
age_group        category
male                int32
female              int32
other               int32
total               int32
dtype: object


In [30]:
# Inspecting the cleaned data
print("\nCleaned Data Info:")
print(df.info())
print("\nMissing Values After Cleaning:")
print(df.isnull().sum())


Cleaned Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3309 entries, 0 to 3308
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   state_name     3309 non-null   object  
 1   district_name  3309 non-null   object  
 2   disability     3309 non-null   object  
 3   age_group      3309 non-null   category
 4   male           3309 non-null   int32   
 5   female         3309 non-null   int32   
 6   other          3309 non-null   int32   
 7   total          3309 non-null   int32   
dtypes: category(1), int32(4), object(3)
memory usage: 132.8+ KB
None

Missing Values After Cleaning:
state_name       0
district_name    0
disability       0
age_group        0
male             0
female           0
other            0
total            0
dtype: int64


In [ ]:
# Saving the cleaned dataset
df.to_csv('cleaned_aggregated_data.csv', index=False)



Cleaned dataset saved as 'cleaned_aggregated_data.csv'
